# Necessary Imports

In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import re
import marcus_keys

# Functions for Web Scraping & Data Processing

In [3]:
def get_city_data(city_lst):
    city_data_df = pd.DataFrame(
        {"city_name": [],
         "country": [],
         "latitude": [],
         "longitude": [],
         "population": []
        }
    )

    for city in city_lst:

        url = f"https://en.wikipedia.org/wiki/{city}"

        headers = {'Accept-Language': 'en-US,en;q=0.8'}
        response = requests.get(url, headers = headers)

        if response.status_code != 200: break

        wiki_city_soup = bs(response.content, "html.parser")

        city_name = wiki_city_soup.select("span.mw-page-title-main")[0].getText()
        country_name = wiki_city_soup.select("table.infobox td.infobox-data")[0].getText()
        latitude = wiki_city_soup.select("span.latitude")[0].getText()
        longitude = wiki_city_soup.select("span.longitude")[0].getText()

        if wiki_city_soup.select_one('th.infobox-header:-soup-contains("Population")'):
            population = wiki_city_soup.select_one('th.infobox-header:-soup-contains("Population")').parent.find_next_sibling().find(text=re.compile(r'\d+'))

        city_df = pd.DataFrame(
            {"city_name": [city_name],
             "country": [country_name],
             "latitude": [latitude],
             "longitude": [longitude],
             "population": [population]
            }
        )

        city_data_df = pd.concat([city_data_df, city_df], ignore_index = True)


        city_data_df["latitude"] = city_data_df["latitude"].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
        city_data_df["longitude"] = city_data_df["longitude"].str.split('″').str[0].str.replace('°', '.', regex=False).str.replace('′', '', regex=False)
        city_data_df["population"] = city_data_df["population"].str.replace(',', '', regex=False)


    return city_data_df

def process_city_data(city_data_raw_df):
    city_data_raw_df["latitude"] = pd.to_numeric(city_data_raw_df["latitude"], errors='coerce')
    city_data_raw_df["longitude"] = pd.to_numeric(city_data_raw_df["longitude"], errors='coerce')
    city_data_raw_df["population"] = pd.to_numeric(city_data_raw_df["population"], errors='coerce')
    
    city_data_raw_df["city_id"] = None
    
    for i in range(0, len(city_data_raw_df["city_id"])):
        if city_data_raw_df.loc[i, "city_name"] == "Berlin":
            city_data_raw_df.loc[i, "city_id"] = "Q64"
        elif city_data_raw_df.loc[i, "city_name"] == "Hamburg":
            city_data_raw_df.loc[i, "city_id"] = "Q1055"
        elif city_data_raw_df.loc[i, "city_name"] == "Stuttgart":
            city_data_raw_df.loc[i, "city_id"] = "Q1022"
        else:
            city_data_raw_df.loc[i, "city_id"] = "Q1718"
            
    return city_data_raw_df

In [4]:
city_lst = ["Berlin", "Hamburg", "Stuttgart", "Duesseldorf"]

In [5]:
city_data_raw_df = get_city_data(city_lst)

In [6]:
city_data_df = process_city_data(city_data_raw_df)

In [7]:
city_data_df

,city_name,country,latitude,longitude,population,city_id
0,Berlin,Germany,52.3112,13.2418,3850809,Q64
1,Hamburg,Germany,NaN,NaN,1906411,Q1055
2,Stuttgart,Germany,48.4639,9.1048,626275,Q1022
3,Düsseldorf,Germany,NaN,NaN,619477,Q1718


# Local MySQL Connection

In [53]:
schema="p5_gans_database"
host="127.0.0.1"
user="root"
password=marcus_keys.my_sql_key
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# AWS RDS MySQL Connection

In [8]:
schema="aws_p5_gans_database"
host="wbs-cs-p5-db.cjdcbdhnueky.eu-north-1.rds.amazonaws.com"
user="mkadmin"
password=marcus_keys.aws_rds_key
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# Uploading the DataFrame into the database

In [9]:
city_data_df.to_sql('city_data_table', 
              if_exists='append', 
              con=con, 
              index=False)

4